# Nonlinear Forecasting

In [1]:
import numpy as np
import nolds
import scipy.io
import mne
import pandas as pd
import seaborn as sns
import pyqtgraph as pg
from scipy.io import loadmat
from scipy import stats
import matplotlib.pyplot as plt
import antropy as ent
from attractors import Attractor
from antropy import higuchi_fd
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.computation import RQAComputation
from pyrqa.time_series import TimeSeries
from pyrqa.result import RQAResult
from pyrqa.opencl import OpenCL
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.image_generator import ImageGenerator
import numpy as np
import matplotlib.pyplot as plt
from antropy import higuchi_fd

In [6]:
# Load data
data = loadmat('/home/vincent/AAA_projects/MVCS/Neuroscience/downsampled/EEG_DS_Struct_0101.mat')
stim_data = pd.read_excel('/home/vincent/AAA_projects/MVCS/Neuroscience/EEG-tES-Chaos-Neural-Net/Stimulation.xlsx')

# Fill null values in 'Sub#' column
stim_data['Sub#'].fillna(method='ffill', inplace=True)

# Drop the first row
stim_data = stim_data.drop(0)

DSamp = data['DSamp']

# Get data parameters
triggers = DSamp[0][0][0]
EEGdata = DSamp[0][0][1]
fs = DSamp[0][0][2][0][0]
fsOld = DSamp[0][0][3][0][0]
time = DSamp[0][0][4][0]
label = DSamp[0][0][5]
nchan = DSamp[0][0][6][0][0]
rate = DSamp[0][0][7][0][0]
npt = DSamp[0][0][8][0][0]
Subj = DSamp[0][0][9][0]
ptrackerPerf = DSamp[0][0][10]
ptrackerTime = DSamp[0][0][11]
ptrackerfs = DSamp[0][0][12][0][0]

# List of unwanted channel names
unwanted_channels = ['BIP1', 'BIP2', 'RESP1']

# Filter out unwanted channels from the label data
filtered_label = [ch for ch in label if ch[0][0] not in unwanted_channels]

# Convert the filtered list back to numpy array and replace the original label
label = np.array(filtered_label, dtype=object)

print(triggers, EEGdata, label, stim_data)

[[(array([[20.428]]), array([[20429]], dtype=uint16), array(['0002'], dtype='<U4'), array([[2]], dtype=uint8), array(['Block Start'], dtype='<U11'), array([], shape=(1, 0), dtype=float64))
  (array([[619.442]]), array([[619443]], dtype=int32), array(['0002'], dtype='<U4'), array([[2]], dtype=uint8), array(['Block Start'], dtype='<U11'), array([], shape=(1, 0), dtype=float64))
  (array([[619.499]]), array([[619500]], dtype=int32), array(['0016'], dtype='<U4'), array([[16]], dtype=uint8), array(['Stim Start'], dtype='<U10'), array(['M30'], dtype='<U3'))
  (array([[654.746]]), array([[654747]], dtype=int32), array(['0032'], dtype='<U4'), array([[32]], dtype=uint8), array(['Stim Stop'], dtype='<U9'), array([], shape=(1, 0), dtype=float64))
  (array([[770.515]]), array([[770516]], dtype=int32), array(['0016'], dtype='<U4'), array([[16]], dtype=uint8), array(['Stim Start'], dtype='<U10'), array(['M30'], dtype='<U3'))
  (array([[805.571]]), array([[805572]], dtype=int32), array(['0032'], dtyp

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert

# Load EEG data
eeg_data = np.loadtxt("eeg_data.txt")

# Define TES stimulation parameters
stimulation_params = {
    "frequency": 10,  # Hz
    "amplitude": 1.5,  # mA
    "duration": 5  # minutes
}

# Calculate phase synchronization for the EEG data and TES stimulation
sampling_rate = 1000  # Hz
eeg_phase = np.angle(hilbert(eeg_data))

stimulation_frequency = stimulation_params["frequency"]
stimulation_duration = stimulation_params["duration"] * 60  # Convert to seconds
stimulation_waveform = stimulation_params["amplitude"] * np.sin(2 * np.pi * stimulation_frequency * np.arange(stimulation_duration))

stimulation_phase = np.angle(hilbert(stimulation_waveform))

# Compute the phase difference between EEG data and TES stimulation
phase_diff = np.angle(np.exp(1j * (stimulation_phase - eeg_phase)))

# Calculate the phase synchronization
sync_strength = np.abs(np.mean(np.exp(1j * phase_diff)))

# Visualize the results
plt.subplot(2, 2, 1)
plt.plot(eeg_data)
plt.title("EEG data")

plt.subplot(2, 2, 2)
plt.plot(stimulation_waveform)
plt.title("TES stimulation")

plt.subplot(2, 2, 3)
plt.plot(phase_diff)
plt.title("Phase difference")

plt.subplot(2, 2, 4)
plt.bar([0, 1], [1 - sync_strength, sync_strength])
plt.xticks([0, 1], ["Desynchronized", "Synchronized"])
plt.title("Phase synchronization")

plt.show()

print("Phase synchronization strength:", sync_strength)

OSError: eeg_data.txt not found.

In [ ]:
import numpy as np
from nolds import lyap_e, pfd
from pyts.approximation import SymbolicAggregateApproximation
from pyts.datasets import load_gunpoint
from pyts.multivariate.image import JointRecurrencePlot
from pyts.multivariate.transformation import MultivariateTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor


# Load EEG data
eeg_data = np.loadtxt("eeg_data.txt")

# Define TES stimulation parameters
stimulation_params = {
    "frequency": 10,  # Hz
    "amplitude": 1.5,  # mA
    "duration": 5  # minutes
}

# Apply TES stimulation
stimulation_duration = stimulation_params["duration"] * 60  # Convert to seconds
stimulation_frequency = stimulation_params["frequency"]
stimulation_amplitude = stimulation_params["amplitude"]
stimulation_waveform = stimulation_amplitude * np.sin(2 * np.pi * stimulation_frequency * np.arange(stimulation_duration))

# Add TES stimulation to the EEG data
eeg_data_with_stimulation = eeg_data + stimulation_waveform

# Calculate the fractal dimension of the EEG data with TES stimulation
fd_with_stimulation = pfd(eeg_data_with_stimulation)

# Apply nonlinear forecasting using Takens' embedding theorem
tau = 10
embedding_dimension = 3
X = np.array([eeg_data_with_stimulation[i:i + embedding_dimension] for i in range(0, len(eeg_data_with_stimulation) - embedding_dimension - tau, tau)])
y = eeg_data_with_stimulation[embedding_dimension + tau:len(eeg_data_with_stimulation):tau]

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use symbolic aggregate approximation to transform the data
n_bins = 4
sax = SymbolicAggregateApproximation(n_bins=n_bins, strategy='uniform')
X_train_sax = sax.fit_transform(X_train)
X_test_sax = sax.transform(X_test)

# Use joint recurrence plot to transform the data
n_neighbors = 2
threshold = 0.1
weights = 'distance'
jrp = JointRecurrencePlot(n_neighbors=n_neighbors, threshold=threshold, weights=weights)
X_train_jrp = jrp.fit_transform(X_train_sax)
X_test_jrp = jrp.transform(X_test_sax)

# Use multivariate transformation to transform the data
mt = MultivariateTransformer(norm_mean=True, norm_std=True)
X_train_mt = mt.fit_transform(X_train_jrp)
X_test_mt = mt.transform(X_test_jrp)

# Train a K-nearest neighbors regression model
n_neighbors = 10
weights = 'distance'
knn = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights)
knn.fit(X_train_mt, y_train)

# Make predictions using the trained model
y_pred = knn.predict(X_test_mt)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error:", mse)